In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qibo
import qibolab
from qibolab.paths import qibolab_folder

from qibolab.pulses import PulseSequence
from qibolab import Platform


print(qibo.__version__)
print(qibolab.__version__)

0.1.9
0.0.1.dev6


In [2]:
runcard = qibolab_folder / "runcards" / "tii_rfsoc4x2.yml"

platform = Platform("tii_rfsoc4x2")


[Qibo 0.1.9|INFO|2023-02-01 10:21:41]: Loading platform tii_rfsoc4x2 from runcard /home/javierserrano/fpgaQCRdrivers/drivers/RFSoC4x2/qibolab/src/qibolab/runcards/tii_rfsoc4x2.yml


In [3]:
platform.connect()
platform.setup()

[Qibo 0.1.9|INFO|2023-02-01 10:21:44]: Connecting to tii_rfsoc4x2 instrument tii_rfsoc4x2.


In [4]:


qd_pulse0 = platform.create_RX_pulse(0, start=0)
qd_pulse1 = platform.create_RX_pulse(0, start=0)
qd_pulse2 = platform.create_RX_pulse(0, start=0)
ro_pulse1 = platform.create_MZ_pulse(0, start=qd_pulse1.duration)
sequence = PulseSequence()
sequence.add(qd_pulse0)
sequence.add(qd_pulse1)
sequence.add(qd_pulse2)
sequence.add(ro_pulse1)

result = platform.execute_pulse_sequence(sequence, nshots=100)

PulseSequence
Pulse(0, 40, 1, 5_440_000_000, 0, Gaussian(5), L3-18_qd, PulseType.DRIVE, 0)
Pulse(0, 40, 1, 5_440_000_000, 0, Gaussian(5), L3-18_qd, PulseType.DRIVE, 0)
Pulse(0, 40, 1, 5_440_000_000, 0, Gaussian(5), L3-18_qd, PulseType.DRIVE, 0)
ReadoutPulse(40, 3000, 0.0156, 7_371_530_000, 0, Rectangular(), L3-18_ro, 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(result)

In [ ]:
start = 10
step = 1
platform.connect()
platform.setup()
lengths = np.arange(start, stop, step)
results = []
for i, length in enumerate(lengths):
    qd_pulse = platform.create_RX_pulse(0, start=0)
    ro_pulse = platform.create_MZ_pulse(0, start=qd_pulse.start)
    qd_pulse.duration = length
    qd_pulse.shape.rel_sigma = length/4

    sequence = PulseSequence()
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)
    platform.setup()
    msr, phase, avgi, avgq = platform.execute_pulse_sequence(sequence)
    results.append((avgi, avgq))

In [ ]:
results = np.array(results)
amp = np.abs(results[:, 0] + 1j * results[:, 1])
plt.figure(figsize=(10, 6))
plt.plot(lengths, amp, 'o-')
plt.title("Length Rabi Oscillation")
plt.xlabel("Drive length (ns)")
plt.ylabel("Amplitude (ADC level)")
plt.savefig("")

In [ ]:
results = np.array(results)
results.shape

In [5]:
runcard = qibolab_folder / "runcards" / "tii_rfsoc4x2.yml"

platform = Platform("tii_rfsoc4x2")
platform.connect()
platform.setup()

start = 7350000000
stop =  7400000000
step = 100000
platform.connect()
platform.setup()
freqs = np.arange(start, stop, step)
length = 40
results = []
for i, freq in enumerate(freqs):
    qd_pulse = platform.create_RX_pulse(0, start=0)
    ro_pulse = platform.create_MZ_pulse(0, start=qd_pulse.start)
    qd_pulse.duration = length
    qd_pulse.shape.rel_sigma = length/4
    ro_pulse.frequency = int(freq)
    platform.settings["instruments"][platform.name]["settings"]["resonator_freq"] = ro_pulse.frequency
    sequence = PulseSequence()
    #sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    platform.setup()
    msr, phase, avgi, avgq = platform.execute_pulse_sequence(sequence)
    results.append((avgi, avgq))

results = np.array(results)
amp = np.abs(results[:, 0] + 1j * results[:, 1])
plt.figure(figsize=(10, 6))
plt.plot(freqs, amp, 'o-')
plt.title("Resonator spectroscopy")
plt.xlabel("Freq")
plt.ylabel("Amplitude (ADC level)")
plt.savefig("")

[Qibo 0.1.9|INFO|2023-02-01 10:24:04]: Loading platform tii_rfsoc4x2 from runcard /home/javierserrano/fpgaQCRdrivers/drivers/RFSoC4x2/qibolab/src/qibolab/runcards/tii_rfsoc4x2.yml
[Qibo 0.1.9|INFO|2023-02-01 10:24:04]: Connecting to tii_rfsoc4x2 instrument tii_rfsoc4x2.


PulseSequence
ReadoutPulse(0, 3000, 0.0156, 7_350_000_000, 0, Rectangular(), L3-18_ro, 0)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
print("Max freq: ",freqs[np.argmax(amp)])


Max freq:  7371400000


In [17]:
shape="Gaussian(5)"
v = shape.split('(')
if v[0] == "Gaussian":
    sigma = v[1].split(')')[0]
print(float(sigma))

5.0
